In [1]:
import sys
import os
import glob
from importlib import reload
import shutil

import pandas as pd
from tqdm import tqdm
import numpy as np
import yaml

In [2]:
pd.set_option("max_colwidth", None)

### Downloading the data from the expriments

In [ ]:
%%bash
mkdir -p data/hp_tuning_results
gcloud config set account $COMMENTS_ON_SEMI_PARAMETRIC_SAMPLING_EMAIL
gsutil -m cp -r gs://$COMMENTS_ON_SEMI_PARAMETRIC_SAMPLING_BUCKET/hp_tuning_* ../data/hp_tuning_results/

### Reading the data

In [3]:
path = "../data/hp_tuning_results"

In [4]:
results = []
for p in tqdm(sorted(glob.glob(os.path.join(path, "*")))): 
    with open(os.path.join(p, 'environment_kwargs'), 'r') as f:
        all_dict = yaml.safe_load(f)
    with open(os.path.join(p, 'regrets'), 'r') as f:
        regrets = yaml.safe_load(f)
        model_name = list(regrets.keys())[0]
    with open(os.path.join(p, f'{model_name}_kwargs'), 'r') as f:
        model_dict = yaml.safe_load(f)
    total_regret = sum(regrets[model_name])
    all_dict.update(model_dict)
    all_dict['total_regret'] = total_regret
    all_dict['cumulative_regret'] = [sum(regrets[model_name][:i]) for i in range(1, len(regrets[model_name])+1)]
    all_dict['path'] = p
    all_dict['regret'] = regrets[model_name]
    all_dict['model_name'] = model_name
    results.append(all_dict)

100%|██████████| 378/378 [24:13<00:00,  3.84s/it]


### Processing the data

In [5]:
df = pd.DataFrame.from_records(results)

In [6]:
df.sort_values(by=['reward_distribution', 'a', 'model_name', 'sigma_1', 'sigma_2', 'sigma_3', 'v', 'seed'], inplace=True)

In [7]:
def parameters(row: pd.core.series.Series) -> str:
    """
    Process parameters into one string so it's easy to display and analyse it
    """
    if row['v'] > 0:
        return 'v = {}'.format(row['v'])
    if row['sigma_1'] > 0:
        return '$\sigma_1$ = {}, $\sigma_2$ = {}, $\sigma_3$ = {}'.format(row['sigma_1'], row['sigma_2'], row['sigma_3'])
    else: 
        return ''

In [8]:
df['parameters'] = df.apply(lambda row: parameters(row), axis=1)

In [9]:
df_grouped_for_all_seeds = df.groupby(['reward_distribution',  'a', 'model_name', 'parameters']).total_regret.agg([np.mean, np.max, np.min]).reset_index().rename(columns={'mean': 'reg_mean',
                                                                                                                                                'amax': 'reg_max', 
                                                                                                                                                'amin': 'reg_min'})
df_grouped_for_all_seeds['hp_rank'] =  df_grouped_for_all_seeds.groupby(['reward_distribution',  'a',  'model_name']).reg_mean.transform(lambda x: x.rank(method='dense'))
df_grouped_for_all_seeds.sort_values(by=['reward_distribution',  'a',  'model_name', 'hp_rank'], inplace=True)

### Displaying the best parameters

In [10]:
df_grouped_for_all_seeds.query("hp_rank == 1")

,reward_distribution,a,model_name,parameters,reg_mean,reg_max,reg_min,hp_rank
0,binomial,0.5,BetaPriorsSampling,,5941.204386,6475.735651,5302.610920,1.0
1,binomial,0.5,GaussianPriorsSampling,,7890.395330,8487.572332,7020.608649,1.0
3,binomial,0.5,LinearGaussianSampling,v = 1.0,1075.894068,1883.257072,356.134152,1.0
5,binomial,0.5,LinearSemiParametricSampling,"$\sigma_1$ = 0.1, $\sigma_2$ = 0.1, $\sigma_3$ = 0.1",824.436089,1056.125711,611.373251,1.0
32,binomial,1.0,BetaPriorsSampling,,4211.215976,5254.014384,2995.814321,1.0
33,binomial,1.0,GaussianPriorsSampling,,12256.829708,13903.871785,10624.124625,1.0
35,binomial,1.0,LinearGaussianSampling,v = 1.0,254.993251,305.614516,157.123289,1.0
38,binomial,1.0,LinearSemiParametricSampling,"$\sigma_1$ = 0.1, $\sigma_2$ = 0.1, $\sigma_3$ = 1.0",301.320199,793.181882,52.636221,1.0
64,normal,0.5,GaussianPriorsSampling,,7963.033596,8630.281851,7044.887675,1.0
66,normal,0.5,LinearGaussianSampling,v = 1.0,1432.603003,1734.267903,1140.338986,1.0
